<a href="https://colab.research.google.com/github/pinkdolphin11/ESAA/blob/main/HW_1014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 04. 텍스트 분류 실습 - 20 뉴스그룹 분류

텍스트 분류 : 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤, 이 학습 모델을 이용해 다른 문서의 분류 예측

텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬의 형태가 되므로, 희소 행렬에 분류를 효과적으로 처리할 수 있는 알고리즘 중 하나인 로지스틱 회귀를 이용해 분류 수행

텍스트 기반 분류

1. 텍스트 정규화 후 피처 벡터화 적용

2. 적합한 머신러닝 알고리즘으로 분류를 학습/예측/평가

카운트 기반, TF-IDF 기반 벡터화를 차례로 적용해 예측 성능을 비교하고, 피처 벡터화를 위한 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝, 사이킷런 Pipeline 객체를 통해 피처 벡터화 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝을 한꺼번에 수행해보자.

### 텍스트 정규화

In [1]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all',random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [ ]:
import pandas as pd
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
# 개별 데이터의 구성 확인
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

기사 내용을 제외하고 뉴스그룹 제목, 작성자 등 다른 정보 제거 - 이러한 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 갖고 있는 경우가 많다. 순수하게 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류하기 위해서는 이러한 정보들을 제거해야 한다.

In [ ]:
# subset 파라미터로 학습 데이터 세트, 테스트 데이터 세트 분리해 내려받기
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용 데이터만 추출, remove = ('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train',remove = ('headers','footers','quotes'),random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset = 'test'으로 학습용 데이터만 추출, remove = ('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test',remove = ('headers','footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

CountVectorizer - 학습 데이터의 텍스트를 피처 벡터화

테스트 데이터의 경우 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 변환(transform)이 이루어져야 한다. 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 변환할 피처 개수가 같아진다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


11314개의 문서에서 피처, 즉 단어가 101631개로 만들어졌다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터, 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegressioni을 이용해 학습/예측/평가
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공한다. 일반적으로 문서 내에 텍스트가 많고, 많은 문서를 가지는 텍스트 분석에서는 카운트 벡터화보다 TF-IDF 벡터화가 좋은 예측 결과를 도출한다.

In [ ]:
# TF-IDF에 더 다양한 파라미터 적용

# stop words 필터링 추가, ngram (1,2)로 변경
tfidf_vect = TfidfVectorizer(stop_words = 'english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


GridSearchCV로 로지스틱 회귀의 하이퍼 파라미터 최적화 수행 - C 파라미터만 변경하면서 최적의 값을 찾고, 이 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행, cv는 3 폴드 세트로 설정
params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid = params, cv=3, scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter : ',grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter :  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


C = 10일 때 가장 좋은 예측 성능을 나타내며, 이를 테스트 세트에 적용했을 때 약 0.703으로 이전보다 약간 성능이 향상되었다.

### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합

Pipeline로 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모댈 코드를 생성할 수 있다. 일반적으로 사이킷런 파이프라인은 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있다.

In [ ]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 만들기
pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),('lr_clf',LogisticRegression(C=10))])

# 별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요 없음
# pipeline의 fit(), predict() 만으로 한번에 피처 벡터화와 ML 학습/예측 가능
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),('lr_clf',LogisticRegression())])

# Pipeline에 기술된 각각의 객체 변수에 언더바 2개를 연달아 붙여 GridSearch에 사용될 파라미터/하이퍼 파라미터 이름과 값 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],'tfidf_vect__max_df':[100,300,700],'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TfidVectorizer 객체의 max_df = 700, ngram_ramge = (1,2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 하이퍼 파라미터 C=10을 적용해 예측 분류를 수행할 때 가장 좋은 검증 세트 성능 수치가 도출되었지만, 이렇게 최적화한 파리미터를 기반으로 테스트 데이터 세트에 대해 예측했을 때의 정확도는 크게 개선되진 않았다.